In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wheat-plant-diseases


In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import os
import pandas as pd

In [19]:
BASE_PATH = '/kaggle/input/wheat-plant-diseases/wheat-plant-diseases/'

train_root = '/kaggle/input/wheat-plant-diseases/data/train'
test_root = '/kaggle/input/wheat-plant-diseases/data/test'
valid_root = '/kaggle/input/wheat-plant-diseases/data/valid'
img_size = (224, 224)
batch_size = 32
SEED = 42
EPOCHS_STAGE_1 = 15
EPOCHS_STAGE_2 = 20

In [20]:
TRAIN_CLASSES = ["Aphid", "Blast", "Mildew", "Smut", "Tan spot"]
VALID_CLASSES = ["aphid_valid", "blast_test_valid", "mildew_valid", "smut_valid", "tan_spot_valid"]
TEST_CLASSES = ["aphid_test", "blast_test", "mildew_test", "smut_test", "tan_spot_test"]
class_names = ["Aphid", "Blast", "Mildew", "Smut", "Tan spot"]
num_classes = len(class_names)

In [21]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.10,
    horizontal_flip=True
)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [22]:
print("--- Initializing Data Generators ---")

try:
    train_data = train_datagen.flow_from_directory(
        train_root, target_size=img_size, classes=TRAIN_CLASSES,
        class_mode="categorical", batch_size=batch_size, shuffle=True, seed=SEED
    )
    val_data = val_test_datagen.flow_from_directory(
        valid_root, target_size=img_size, classes=VALID_CLASSES,
        class_mode="categorical", batch_size=batch_size, shuffle=False
    )
    test_data = val_test_datagen.flow_from_directory(
        test_root, target_size=img_size, classes=TEST_CLASSES,
        class_mode="categorical", batch_size=batch_size, shuffle=False
    )
except Exception as e:
    print(f"Error loading data: {e}")
    raise RuntimeError("Could not load data. Check BASE_PATH and folder names.")

print("Data Loading Complete.")

--- Initializing Data Generators ---
Found 4711 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Data Loading Complete.


In [23]:
print("\n--- Building DenseNet121 Model ---")

base_model = DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(img_size[0], img_size[1], 3)
)
base_model.trainable = False # Base Model ফ্রিজ করা

inputs = layers.Input(shape=(img_size[0], img_size[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
# বন্ধুর কোড অনুযায়ী কাস্টম ক্লাসিফিকেশন লেয়ার
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.4)(x) # 0.5 এর বদলে 0.4 ব্যবহার করা হলো (যেহেতু আপনার বন্ধুর কোডে 0.4 ছিল)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.summary()


--- Building DenseNet121 Model ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,694,917 (29.35 MB)

 Trainable params: 657,413 (2.51 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [24]:
callbacks = [
    # সেরা মডেল সেভ করা
    ModelCheckpoint("best_densenet121.keras", monitor="val_accuracy", save_best_only=True, verbose=1),
    # ওভারফিটিং হলে ট্রেনিং থামানো
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1),
    # ভ্যালিডেশন লস স্থির থাকলে লার্নিং রেট কমানো
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, min_lr=1e-7, verbose=1),
]

In [25]:
print("\n--- Stage 1: Feature Extraction (Fast Learning) ---")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), # 0.001
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
start_time_1 = tf.timestamp()
history1 = model.fit(
    train_data,
    validation_data=val_data,    
    epochs=EPOCHS_STAGE_1,
    callbacks=callbacks
)
end_time_1 = tf.timestamp()


--- Stage 1: Feature Extraction (Fast Learning) ---


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1765400649.981576     141 service.cc:148] XLA service 0x7fdb340026c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765400649.982458     141 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765400649.982481     141 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765400652.820817     141 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/148 ━━━━━━━━━━━━━━━━━━━━ 1:08:14 28s/step - accuracy: 0.2188 - loss: 1.7835

I0000 00:00:1765400664.771647     141 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 739ms/step - accuracy: 0.5749 - loss: 1.0892
Epoch 1: val_accuracy improved from -inf to 0.75000, saving model to best_densenet121.keras
148/148 ━━━━━━━━━━━━━━━━━━━━ 158s 888ms/step - accuracy: 0.5756 - loss: 1.0875 - val_accuracy: 0.7500 - val_loss: 0.5814 - learning_rate: 0.0010
Epoch 2/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.7788 - loss: 0.5817
Epoch 2: val_accuracy improved from 0.75000 to 0.83000, saving model to best_densenet121.keras
148/148 ━━━━━━━━━━━━━━━━━━━━ 78s 526ms/step - accuracy: 0.7788 - loss: 0.5817 - val_accuracy: 0.8300 - val_loss: 0.4413 - learning_rate: 0.0010
Epoch 3/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 507ms/step - accuracy: 0.8031 - loss: 0.5310
Epoch 3: val_accuracy did not improve from 0.83000
148/148 ━━━━━━━━━━━━━━━━━━━━ 76s 515ms/step - accuracy: 0.8032 - loss: 0.5308 - val_accuracy: 0.8000 - val_loss: 0.4751 - learning_rate: 0.0010
Epoch 4/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - accuracy: 0.8238 - l

In [26]:
print("\n--- Stage 2: Fine-Tuning (Slow Learning) ---")

# বেস মডেলের শেষ 40টি লেয়ার আনফ্রিজ করা (বন্ধুর কোড অনুযায়ী)
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

# দ্বিতীয়বার কম্পাইল করা (লার্নিং রেট অনেক কমিয়ে)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # 0.00001
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

start_time_2 = tf.timestamp()
history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_STAGE_2, # 20টি Epochs
    callbacks=callbacks
)
end_time_2 = tf.timestamp()


--- Stage 2: Fine-Tuning (Slow Learning) ---
Epoch 1/20
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.7874 - loss: 0.7166
Epoch 1: val_accuracy did not improve from 0.96000
148/148 ━━━━━━━━━━━━━━━━━━━━ 135s 706ms/step - accuracy: 0.7875 - loss: 0.7157 - val_accuracy: 0.9000 - val_loss: 0.2089 - learning_rate: 1.0000e-05
Epoch 2/20
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 509ms/step - accuracy: 0.8712 - loss: 0.3927
Epoch 2: val_accuracy did not improve from 0.96000
148/148 ━━━━━━━━━━━━━━━━━━━━ 76s 516ms/step - accuracy: 0.8713 - loss: 0.3925 - val_accuracy: 0.9000 - val_loss: 0.2129 - learning_rate: 1.0000e-05
Epoch 3/20
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy: 0.8760 - loss: 0.3387
Epoch 3: val_accuracy did not improve from 0.96000
148/148 ━━━━━━━━━━━━━━━━━━━━ 76s 514ms/step - accuracy: 0.8761 - loss: 0.3386 - val_accuracy: 0.9000 - val_loss: 0.2047 - learning_rate: 1.0000e-05
Epoch 4/20
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy: 0.9029 - loss: 0.2818
Epoc

In [29]:
best_model = tf.keras.models.load_model("best_densenet121.keras")
total_training_time = (end_time_1 - start_time_1 + end_time_2 - start_time_2) 
total_training_time_s = tf.get_static_value(total_training_time)



In [30]:
# টেস্ট ডেটার মূল্যায়ন
test_data.reset() # জেনারেটর রিসেট করা
test_loss, test_acc = best_model.evaluate(test_data, verbose=1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.9406 - loss: 0.1868 


In [31]:
preds = best_model.predict(test_data, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = test_data.classes

8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step 


In [32]:
print("\n--- FINAL CLASSIFICATION REPORT ---")
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0)
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

print(f"\nFinal Test Accuracy: {round(float(test_acc), 4)}")
print(f"Total Training Time: {total_training_time_s:.2f} seconds")


--- FINAL CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

       Aphid       0.94      0.90      0.92        50
       Blast       0.94      1.00      0.97        50
      Mildew       0.94      0.96      0.95        50
        Smut       0.98      1.00      0.99        50
    Tan spot       0.91      0.86      0.89        50

    accuracy                           0.94       250
   macro avg       0.94      0.94      0.94       250
weighted avg       0.94      0.94      0.94       250


Final Test Accuracy: 0.944
Total Training Time: 2354.07 seconds


In [33]:
model_name = "DenseNet121"

metrics_summary = {
    'Model': model_name,
    'Accuracy': test_acc,
    'Precision': report['weighted avg']['precision'], 
    'Recall': report['weighted avg']['recall'],
    'F1-Score': report['weighted avg']['f1-score'],
    'Training Time (s)': total_training_time_s,
}

df_metrics = pd.DataFrame([metrics_summary])
results_file = 'model_performance_summary.csv'

In [34]:
if not os.path.exists(results_file):
    df_metrics.to_csv(results_file, index=False)
else:
    df_metrics.to_csv(results_file, mode='a', header=False, index=False)
    
print(f"\nMetrics saved to {results_file}")
print("DenseNet121 workflow complete!")


Metrics saved to model_performance_summary.csv
DenseNet121 workflow complete!
